In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.cuda
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(1)    # reproducible
os.chdir(r'C:\Users\pbhavsa\OneDrive - MORNINGSTAR INC\git\msai')

In [2]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_capability(0))
print(torch.cuda.is_available())
print(torch.cuda.max_memory_allocated(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(f'Running on {device}')

1
Quadro M1200
(5, 0)
True
0
Running on cpu


In [3]:
df = pd.read_csv('data/data_p.tsv', sep='\t', header=None)
df.head()

,0,1,2,3,4
0,100126,how do i achieve my sphr certificate,Applications can be completed online at the Hu...,0,7
1,100075,how much fat is in a bubba burger,"Meat and meat products. Beef. Bubba Burgers, G...",0,9
2,1004,do spiders eat other animals,Spider - Description. 1 1: pedipalp 2: tricho...,0,5
3,100490,definition of chiropractic,chiropractic [(keye-ruh-prak-tik)] A system of...,0,7
4,10283,the ________ is divided into the parasympathet...,What is the sympathetic nervous system? The sy...,0,3


In [4]:
y = torch.LongTensor(df.iloc[:, 3].values).to(device)
y

tensor([0, 0, 0,  ..., 0, 1, 1])

In [5]:
query = np.load('data/query_vectors.npy')
passage = np.load('data/passage_vectors.npy')
x = torch.FloatTensor(np.append(query, passage, axis=1)).to(device)
x.shape

torch.Size([1048376, 100])

In [6]:
# torch can only train on Variable, so convert them to Variable
# x, y = Variable(x), Variable(y)

class Data(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.len = self.x.shape[0]
        
    def __getitem__(self, index):    
        return self.x[index],self.y[index]
    
    def __len__(self):
        return self.len
    
dataset=Data(x,y)
trainloader = DataLoader(dataset=dataset, batch_size=512, shuffle=True, num_workers=0)

In [7]:
class Net(nn.Module):
    def __init__(self, n_feature, n_output):
        super(Net, self).__init__()
        h1 = 80
        h2 = 50
        h3 = 25
        self.h1 = nn.Linear(n_feature, h1)   # hidden layer
        self.h2 = nn.Linear(h1, h2)   # hidden layer
        self.h3 = nn.Linear(h2, h3)   # hidden layer
        self.out = nn.Linear(h3, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.h1(x))      
        x = F.relu(self.h2(x))
        x = F.relu(self.h3(x))
        x = self.out(x)
        return x

In [8]:
model = Net(n_feature=100, n_output=2).to(device)     # define the network
print(model)  # net architecture

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
optimizer = torch.optim.Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

Net(
  (h1): Linear(in_features=100, out_features=80, bias=True)
  (h2): Linear(in_features=80, out_features=50, bias=True)
  (h3): Linear(in_features=50, out_features=25, bias=True)
  (out): Linear(in_features=25, out_features=2, bias=True)
)


In [9]:
count = dataset.len
print(dataset.len)
        
for t in range(100):
    i=0
    for x_b,y_b in tqdm(trainloader):
        out = model(x_b)                 # input x and predict based on x
        loss = loss_func(out, y_b)     # must be (1. nn output, 2. target), the target label is NOT one-hotted

        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients
        i+=1
        
    if t % 1 == 0:
        out = model(x)  
        _, prediction = torch.max(F.softmax(out), 1)
        device = 'cpu'
        pred_y = prediction.to(device).data.numpy().squeeze()
        target_y = y.to(device).data.numpy()
        accuracy = sum(pred_y == target_y)/count
        print(accuracy)

1048376


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [00:17<00:00, 116.13it/s]
C:\Users\pbhavsa\AppData\Local\Continuum\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.5979142979236457


 24%|█████████████████████████████████████                                                                                                                       | 487/2048 [00:04<00:13, 114.24it/s]


KeyboardInterrupt: 